In [33]:
import numpy as np

delta_1 = 0.1
delta_2 = 0.1
eps_1 = 0.3
eps_2 = 0.3
ro_1 = 0.1
ro_2 = 0.1
tau = 0.1
g_ma = 0.9
g_mi = 0.1



def m(x,y):
    ly = len(y)
    lx = len(x)
    down = np.array([[np.nan] * (ly+1)] * (lx+1))
    right = np.array([[np.nan] * (ly+1)] * (lx+1))
    broadway = np.array([[np.nan] * (ly+1)] * (lx+1))
    right[0, 1] = delta_1
    down[1, 0] = delta_2
    right[1, 0] = 0
    down[0, 1] = 0
    broadway[0, 1] = 0
    broadway[1, 0] = 0
    for j in range(2, ly+1):
        right[0, j] = right[0, j-1] * eps_1
        down[0, j] = 0
        broadway[0, j] = 0
    for i in range(2, lx+1):
        down[i, 0] = down[i-1, 0] * eps_2
        right[i,0] = 0
        broadway[i,0] = 0
    broadway[1,1] = 1 - delta_1 - delta_2
    for j in range(1,ly+1):
        down[1, j] = right[0, j] * ro_2    
    for i in range(1,lx+1):
        right[i, 1] = down[i, 0] * ro_1
    if x[0] == y[0]:
        g = g_ma
    else:
        g = g_mi
    broadway[1, 1] = (1 - eps_1 - ro_2 - tau) * g
    return down, right, broadway, lx, ly



def f(down, right, broadway, lx, ly):
    for i in range(1, lx+1):
        for j in range(1, ly+1):
            if x[i-1] == y[j-1]:
                g = g_ma
            else:
                g = g_mi
            if i*j != 1:
                broadway[i, j] = broadway[i-1, j-1] * (1 - delta_1 - delta_2-tau) * g + down[i-1, j-1] * (1 - ro_1 - eps_2 - tau) * g + right[i-1, j-1]*(1 - ro_2 - eps_1 - tau) * g
                down[i, j] = broadway[i-1,j]*(delta_2) + down[i-1, j]*(eps_2)+right[i-1,j]*(ro_2)
                right[i, j] = broadway[i,j-1]*(delta_1) + down[i, j-1] * (ro_1)+right[i,j-1]*(eps_1) 
    return down, right, broadway


def m_b(down, right, broadway, lx, ly):
    down_b = np.array([[np.nan] * (ly+1)] * (lx+1))
    right_b = np.array([[np.nan] * (ly+1)] * (lx+1))
    broadway_b = np.array([[np.nan] * (ly+1)] * (lx+1))
    down_b[-1,-1] = 1
    right_b[-1,-1] = 1
    broadway_b[-1,-1] = 1
    for j in range(ly-1, -1, -1):
        broadway_b[lx,j] = right_b[lx, j+1] * (delta_1)
        down_b[lx,j] = right_b[lx, j+1] * (ro_1)
        right_b[lx,j] = right_b[lx, j+1] * (eps_1)
    for i in range(lx-1, -1, -1):
        broadway_b[i,ly] = down_b[i+1,ly] * (delta_2)
        down_b[i,ly] = down_b[i+1,ly] * (eps_2)
        right_b[i,ly] = down_b[i+1,ly] * (ro_2)
    return down_b, right_b, broadway_b
    
    
    
    
def f_b(down_b, right_b, broadway_b, lx, ly):
    for i in range(lx-1, -1, -1):
        for j in range(ly-1, -1, -1):
                if x[i] == y[j]:
                    g = g_ma
                else:
                    g = g_mi
                broadway_b[i, j] = broadway_b[i+1, j+1] * (1 - delta_1 - delta_2 - tau) * g + right_b[i, j+1] * (delta_1) + down_b[i+1, j] * (delta_2)
                down_b[i, j] = broadway_b[i+1, j+1] * (1-ro_1 - eps_2-tau) * g + right_b[i, j+1] * (ro_1) + down_b[i+1, j]*(eps_2)
                right_b[i, j] = broadway_b[i+1, j+1]*(1 - ro_2 - eps_1-tau) * g + right_b[i, j+1]*(eps_1) + down_b[i+1, j]*(ro_2)
    return down_b, right_b, broadway_b



def main():
    down, right, broadway, lx, ly = m(x,y)
    down, right, broadway = f(down, right, broadway, lx, ly)
    down_b, right_b, broadway_b = m_b(down, right, broadway, lx, ly)
    down_b, right_b, broadway_b=f_b(down_b, right_b, broadway_b, lx, ly) 
    down_p = np.array([[np.nan] * (ly+1)] * (lx+1))
    right_p = np.array([[np.nan] * (ly+1)] * (lx+1))
    broadway_p = np.array([[np.nan] * (ly+1)] * (lx+1))
    sum_p = np.array([[np.nan] * (ly+1)] * (lx+1))
    for i in range(0, lx+1):
            for j in range(0, ly+1):
                broadway_p[i, j] = broadway[i, j] * broadway_b[i, j]
                right_p[i, j] = right[i, j] * right_b[i, j]
                down_p[i, j] = down[i, j] * down_b[i, j]
                sum_p[i, j] = broadway_p[i, j] + right_p[i, j] + down_p[i,j ]
                broadway_p[i, j] = broadway_p[i, j] / sum_p[i, j]
                right_p[i, j] = right_p[i, j] / sum_p[i, j]
                down_p[i, j] = down_p[i, j] / sum_p[i, j]
    return down_p, right_p, broadway_p
                

                
               


x='AGA'
y='AGAGA'
down_p, right_p, broadway_p=main()
print('G down =')
print(down_p)
print()
print('G right =')
print(right_p)
print()
print('M =')
print(broadway_p)
f2 = open("h5_HMM.txt", "w")
f2.write("G down =\n")
np.savetxt(f2, down_p, fmt='%.2f')
f2.write("\n")
f2.write("G right =\n")
np.savetxt(f2, right_p, fmt='%.2f')
f2.write("\n")
f2.write("M =\n")
np.savetxt(f2, broadway_p, fmt='%.2f')
f2.write("\n")
f2.close()


print()
print()
x='ATAGCTACGAC'
y='TGCTAGCTAGC'                
down_p, right_p, broadway_p=main()          
print('G down =')
print(down_p)
print()
print('G right =')
print(right_p)
print()
print('M =')
print(broadway_p)


f2 = open("h5_HMM.txt", "a")
f2.write("G down =\n")
np.savetxt(f2, down_p, fmt='%.2f')
f2.write("\n")
f2.write("G right =\n")
np.savetxt(f2, right_p, fmt='%.2f')
f2.write("\n")
f2.write("M =\n")
np.savetxt(f2, broadway_p, fmt='%.2f')
f2.close()

#np.savetxt('test.out', (down_p,right_p,broadway_p))
#np.savetxt('result.txt', (down_p,right_p,broadway_p))
#np.savetxt('result.txt', right_p, fmt='%.2f')
#np.savetxt('result.txt', broadway_p, fmt='%.2f')

G down =
[[       nan 0.         0.         0.         0.         0.        ]
 [1.         0.01803395 0.02504478 0.02621136 0.09257416 0.07238606]
 [1.         0.79666866 0.01830752 0.03769457 0.02300474 0.16614699]
 [1.         0.48895899 0.71829075 0.0213016  0.04209614 0.02728489]]

G right =
[[       nan 1.         1.         1.         1.         1.        ]
 [0.         0.03005275 0.93808358 0.45141782 0.848696   0.56568365]
 [0.         0.12602031 0.03517124 0.92945409 0.3174654  0.77619401]
 [0.         0.0851735  0.21501243 0.0639048  0.92877666 0.30189565]]

M =
[[       nan 0.         0.         0.         0.         0.        ]
 [0.         0.95191329 0.03687164 0.52237082 0.05872984 0.36193029]
 [0.         0.07731104 0.94652124 0.03285134 0.65952986 0.057659  ]
 [0.         0.42586751 0.06669681 0.91479361 0.0291272  0.67081946]]


G down =
[[       nan 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [1.